# California Clear Sterile Compound License Enrichment

## Notebook Config

In [53]:
import pandas as pd
import re 
import numpy as np



# Set pandas to display more rows/columns for wider dataframes
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 100)     # Adjust rows
pd.set_option('display.width', 1000)       # Increase width for better visibility

In [54]:
# Load Data
pharmacies = pd.read_csv("all_pharmacies.csv", dtype=str)
# Convert ExpiratioN Date to Date
pharmacies['Expiration Date'] = pd.to_datetime(pharmacies['Expiration Date'])

In [55]:
zips = pd.read_csv("zip_lat_long.csv", dtype=str)

## Enrich Location Data

In [56]:
def get_lat_lon(df_to_edit: pd.DataFrame, df_with_zips: pd.DataFrame) -> pd.DataFrame:
    merged_df = df_to_edit.merge(df_with_zips, on='Zip', how="left")
    return merged_df

In [57]:
def update_state_to_abbreviation(df):

    state_name_to_abbrev = {
        'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
        'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
        'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
        'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
        'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO',
        'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
        'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH',
        'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
        'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
        'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY',
    }
    df['State'] = df['State'].map(state_name_to_abbrev)
    return df


In [58]:
def replace_out_of_state_with_nan(df):
    df['County'] = df['County'].replace('OUT OF STATE', np.nan)
    return df

In [59]:
pharmacies = get_lat_lon(pharmacies, zips)
pharmacies = update_state_to_abbreviation(pharmacies)
pharmacies = replace_out_of_state_with_nan(pharmacies)
pharmacies.head()

,Pharmacy Name,License Number,License Type,License Status,Expiration Date,City,State,County,Zip,LAT,LONG
0,986 INFUSION PHARMACY #1 INC,LSC 101414,Sterile Compounding Pharmacy,CLEAR,2025-07-01,ANAHEIM,CA,ORANGE,92807,33.848733,-117.788357
1,A & O SPECIALTY PHARMACY,LSC 101817,Sterile Compounding Pharmacy,CLEAR,2025-03-01,SALINAS,CA,MONTEREY,93901,36.636344,-121.623978
2,ACCREDO HEALTH GROUP INC,NSC 101280,Nonresident Sterile Compounding Pharmacy,CLEAR,2025-12-01,WARRENDALE,PA,NaN,15086,40.674947,-80.106364
3,ACCREDO HEALTH GROUP INC,NSC 101279,Nonresident Sterile Compounding Pharmacy,CLEAR,2025-12-01,ORLANDO,FL,NaN,32822,28.489898,-81.290154
4,ADVANCE CARE PHARMACY,LSC 99611,Sterile Compounding Pharmacy,CLEAR,2025-07-01,ESCONDIDO,CA,SAN DIEGO,92025,33.102005,-117.080419


## 

## Identify & Remove Duplicate Licenses

In [60]:
def remove_previous_names(df):

    # Remove rows where 'Pharmacy Name' contains '(Previous Name)'
    df = df[~df['Pharmacy Name'].str.contains(r'\(Previous Name\)', na=False)]
    
    return df

In [61]:
pharmacies = remove_previous_names(pharmacies)

In [62]:
# Show rows with duplicate License Numbers
duplicates = pharmacies[pharmacies.duplicated(subset='License Number', keep=False)]
duplicates


,Pharmacy Name,License Number,License Type,License Status,Expiration Date,City,State,County,Zip,LAT,LONG


In [63]:
previous_name_rows = pharmacies[pharmacies['Pharmacy Name'].str.contains(r'\(Previous Name\)', na=False)]
previous_name_rows

,Pharmacy Name,License Number,License Type,License Status,Expiration Date,City,State,County,Zip,LAT,LONG


## Enrich Pharmacies with Category & Typing Data

In [64]:
def identify_hospitals(df: pd.DataFrame) -> pd.DataFrame:
    # List of keywords for identifying hospitals and medical facilities
    hospital_keywords = [
        'ADVENTIST HEALTH',
        'BOWYER PHARMACY',
        'CAMPUS',
        'CANCER CENTER',
        'CARDINAL HEALTH',
        'CARE PAVILION',
        'CEDARS-SANAI',
        'CEDARS-SINAI',
        'CENTER FOR',
        'CITY OF HOPE',
        'CLINIC',
        'CONVERSIO HEALTH',
        'CORRECTIONAL FACILITY',
        'COUNTY ONCOLOGY',
        'CRMC',
        'EMANATE HEALTH',
        'HEALTH CENTER',
        'HEALTH SERVICES',
        'HEALTH SYSTEM',
        'HEALTHCARE DISTRICT',
        'HIGHLAND CARE PAVILION',
        'HOAG',
        'HOSPITAL',
        'HOSPTIAL',
        'HSP',
        'KAISER',
        'KAWEAH HEALTH',
        'LPCH',
        'LUNDQUIST INSTITUTE',
        'MED CTR',
        'MEDICAL CENTER',
        'MEMORIAL',
        'MOUNTAIN VIEW PHARMACEUTICALS',
        'OF USC',
        'ONCOLOGY CENTER',
        'OUTPATIENT',
        'PENTEC HEALTH',
        'PRIME HEALTHCARE',
        'PROVIDENCE ONCOLOGY',
        'RADY CHILDREN\'S',
        'REGIONAL',
        'REHABILITATION',
        'RIVER\'S EDGE',
        'RUHS',
        'SCRIPPS',
        'SHRINERS',
        'SOLEO HEALTH INC',
        'SONOMA SPECIALTY PHARMACY',
        'STANFORD HEALTH CARE',
        'SUTTER LAKESIDE',
        'TODD CANCER PAVILION',
        'UC DAVIS',
        'UCI HEALTH',
        'CHILDRENS VILLAGE PHARMACY',
        'CONSONUS PHARMACY SERVICES CA NORTH LLC',
        'CRESCENT HEALTHCARE INC DBA CRESCENT HEALTHCARE',
        'GMT PHARMACY',
        'HIGHLAND CARE PAVILLION PHARMACY',
        'LA JOLLA DISCOUNT PHARMACY',
        'PACIFIC WEST PHARMACY INC',
        'SKILLED NURSING PHARMACY'
    ]

    # Create a regex pattern to match any of the hospital keywords
    keywords_pattern = '|'.join([re.escape(keyword) for keyword in hospital_keywords])

    # Use .loc[] to safely assign "Medical Facility" where conditions are met
    df.loc[df['Pharmacy Name'].str.contains(keywords_pattern, case=False, regex=True, na=False), 'Facility Type'] = 'Medical Facility'

    return df


def identify_veterinary_only(df: pd.DataFrame) -> pd.DataFrame:
    # List of keywords for identifying veterinary-only pharmacies
    vet_keywords = [
        'PETNET PHARMACEUTICAL',
        'PROLAB PHARMACY',
        'SPECIALTY VETERINARY PHARMACY',
        'WEDGEWOOD PHARMACY LLC',
        'BCP VETERINARY PHARMACY',
        'COVETRUS NORTH AMERICA, LLC']

    # Use .loc[] to safely assign "Veterinary Only" where conditions are met
    df.loc[df['Pharmacy Name'].apply(lambda x: any(keyword in x for keyword in vet_keywords)), 'Facility Type'] = 'Veterinary Only'

    return df


def identify_infusion_centers(df: pd.DataFrame) -> pd.DataFrame:
    # List of keywords for identifying infusion centers
    infusion_center_search_terms = [
        'INFUSION',
        'HOSPICARE',
        'INTEGRATED CARE SYSTEMS',
        'IV LEAGUE',
        'KABAFUSION',
        'MARSHALL MEDICAL ONCOLOGY PHARMACY',
        'MODEL DRUG',
        'NUTRISHARE',
        'OMNICARE',
        'OPTION CARE',
        'OPTUM',
        'OSO HOME CARE',
        'OWL REXALL DRUG',
        'PLAZA HOME CARE',
        'POLARIS PHARMACY SERVICES',
        'PREFERRED EXCELLENT CARE',
        'QUADRACARE',
        'SCRIPTS',
        'SHARP CENTRAL PHARMACY SERVICES',
        'TRINITY CARE RX',
        'VIVE RX',
        'ACCREDO HEALTH GROUP INC',
        'ALERACARE PHARMACY CALIFORNIA',
        'AMERIPHARMA',
        'BIOMED CALIFORNIA INC',
        'BROOKS HEALTH CARE',
        'CALIFORNIA SPECIALTY PHARMACY LLC',
        'ELEVARE SPECIALTY SERVICES',
        'FIRST CALL PHARMACY LLC',
        'CVS/SPECIALTY OR CARELONRX SPECIALTY PHARMACY',
        'CORAM HEALTHCARE CORPORATION OF NEVADA DBA CORAM CVS/SPECIALTY IN',
        'CAREMARK ILLINOIS SPECIALTY PHARMACY, LLC DBA CVS/SPECIALTY OR',
        'ALLIANCERX WALGREENS PHARMACY',
        'PHARMERICA',
        'WELLSCRIPT'
    ]
   
    # Create a regex pattern to match any of the infusion center keywords
    pattern = '|'.join(infusion_center_search_terms)

    # Use .loc[] to safely assign "Infusion Center" based on conditions in License Type or Pharmacy Name
    df.loc[df['License Type'].str.contains('Infusion', case=False, na=False) | 
           df['Pharmacy Name'].str.contains(pattern, case=False, na=False), 'Facility Type'] = 'Infusion Center'

    return df

def identify_research_centers(df: pd.DataFrame) -> pd.DataFrame:
    # List of keywords for identifying veterinary-only pharmacies
    research_keywords = [
        'UCSD RESEARCH PHARMACY'
    ]

    # Use .loc[] to safely assign "Veterinary Only" where conditions are met
    df.loc[df['Pharmacy Name'].apply(lambda x: any(keyword in x for keyword in research_keywords)), 'Facility Type'] = 'Research Center'

    return df

In [65]:
# List of keywords for identifying veterinary-only pharmacies
verified_pharmacies = {
'A & O SPECIALTY PHARMACY' : {
'specialties' : ['Hormone Replacement Therapy (HRT)', 'Veterinary', 'Dermatology', 'Pain Management', 'Gastroenterology', 'Ophthalmology', 'Oncology'],
'conditions' : ['Erectile Dysfunction', 'Thyroid Dysfunction'],
'registered outsourcer' : False,
'accreditations' : ['PCCA', 'IACP', 'PCAB'],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://www.aocompounding.com'
},
'ADVANCE CARE PHARMACY' : {
'specialties' : ['Hospice', 'Long Term Care', 'Pain Management', 'Antibiotics', 'Total Parenteral Nutrition (TPN)'],
'conditions' : ['Terminally Ill',],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://www.advancecarepharmacy.com'
},
'AMBROSIA HEALTHCARE INC' : {
'specialties' : ['Veterinary', 'Pain Management', 'Hospice', 'Hormone Replacement Therapy (HRT)', 'Long Term Care'],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : ['IACP'],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://www.ambrosiahc.com'
},
'BOOTHWYN PHARMACY LLC' : {
'specialties' : ['Veterinary', 'Ophthalmology', 'Weight Loss', 'Hormone Replacement Therapy (HRT)', 'Dermatology', 'Pediatrics', 'Pain Management', 'Gastroenterology', 'Podiatry'],
'conditions' : ['Infertility', 'Glaucoma', 'Dry Eye Syndrome', 'Acne', 'Psoriasis', 'Eczema'],
'registered outsourcer' : False,
'accreditations' : ['PCAB', 'ACHC', 'LegitScript'],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://boothwyn.com'
},
'CD PHARMACY LLC DBA RED ROCK PHARMACY' : {
'specialties' : ['Hospice', 'Long Term Care', 'Weight Loss'],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://www.redrockrx.com'
},
'CENTRAL ADMIXTURE PHARMACY SERVICES INC' : {
'specialties' : ['Total Parenteral Nutrition (TPN)', 'Anesthesia'],
'conditions' : ['Cardioplegia'],
'registered outsourcer' : True,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://www.capspharmacy.com/en.html'
},
'CHEMIQUE PHARMACEUTICALS INC' : {
'specialties' : ['IV Vitamin Therapy', 'Total Parenteral Nutrition (TPN)', 'Weight Loss', 'Antibiotics', 'Hormone Replacement Therapy (HRT)', 'Veterinary', 'Oncology', 'Pain Management'],
'conditions' : ['Anemia', 'Nausea', 'Diabetes', 'Hypertension'],
'registered outsourcer' : False,
'accreditations' : ['American Association of Homeopathic Pharmacists', 'ACHC', 'ASC' , 'APhA', 'ASHP', 'ASPEN', 'NHIA', 'PDA'],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://chemiquepharm.com'
},
'CITRUS PHARMACY' : {
'specialties' : ['Long Term Care'],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://citrusrx.com'
},
'CLINISHARE PHARMACY' : {
'specialties' : [''],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://www.rxless.com/pharmacies/california/westhills/clinishare-pharmacy/23719-Roscoe-Blvd/1477638765'
},
'COAST QUALITY PHARMACY LLC D/B/A ANAZAOHEALTH' : {
'specialties' : ['IV Vitamin Therapy', 'Pain Management', 'Weight Management', 'Hormone Replacement Therapy (HRT)', 'Hair Care', 'Nuclear Medicine', 'Anti-Aging', 'Sexual Health', 'Aesthetics'],
'conditions' : [''],
'registered outsourcer' : True,
'accreditations' : ['LegitScript'],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://www.anazaohealth.com'
},
'CURE STAT RX' : {
'specialties' : ['IV Vitamin Therapy', 'Oncology', 'Hematology', 'Cardiology', 'Immunotherapy', 'Total Parenteral Nutrition (TPN)', 'Chronic Disease', 'Ophthalmology', 'Veterinary'],
'conditions' : ['Electrolyte Imbalance', 'Appetite', 'Infertility'],
'registered outsourcer' : False,
'accreditations' : ['URAC', 'ACHC', 'NHIA', 'PCAB'],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://curestatrx.com'
},
'DEL\'S PHARMACY I' : {
'specialties' : ['Long Term Care', 'Hospice', 'Total Parenteral Nutrition (TPN)'],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://delspharmacy.com'
},
'DIAMOND DRUGS INC' : {
'specialties' : ['Long Term Care', 'Correctional Populations', ],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://www.diamondpharmacy.com'
},
'DWAYNE\'S FRIENDLY PHARMACY' : {
'specialties' : [''],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://dwaynespharmacy.com'
},
'DYNAMIC RX LABS LLC' : {
'specialties' : ['IV Vitamin Therapy', 'Weight Loss', 'Hormone Replacement Therapy (HRT)', 'Hair Care', 'Sexual Health'],
'conditions' : ['Erectile Dysfunction', 'Premature Ejaculation'],
'registered outsourcer' : False,
'accreditations' : ['LegitScript'],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://dynamicrxlabs.com'
},
'ENCINO PHARMACY' : {
'specialties' : [''],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://www.encinopharmacy.com'
},
'ENOVEX PHARMACY' : {
'specialties' : ['Pain Management', 'Hormone Replacement Therapy (HRT)', 'Veterinary', 'Dermatology'],
'conditions' : ['Arthritis'],
'registered outsourcer' : False,
'accreditations' : ['PCAB', 'PCCA', 'A4PC'],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://enovexrx.com'
},
'FIRESIDE PHARMACY' : {
'specialties' : ['IV Vitamin Therapy', 'Hormone Replacement Therapy (HRT)', 'Sexual Health', 'Weight Loss', 'Anti-Aging', 'Dentistry', 'Dermatology', 'Ear, Nose, and Throat', 'Pain Management', 'Palliative Care', 'Pediatrics', 'Podiatry', 'Sports Medicine', 'Veterinary', 'Wound Care'],
'conditions' : ['Erectile Dysfunction'],
'registered outsourcer' : False,
'accreditations' : ['PCCA'],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://firesidepharmacy.com'
},
'HOPEWELL PHARMACY' : { 
'specialties' : [''],
'conditions' : ['Autism'],
'registered outsourcer' : False,
'accreditations' : ['PCAB', 'ACHC'],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://www.hopewellrx.com'
},
'HYGEIA LTC PHARMACY' : {
'specialties' : ['Long Term Care', 'Hospice', 'Adult Residential Facility (ARF)', 'Residential Care Facility for the ELderly (RCFE)', 'Intermediate Care Facility'],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://hygeialtc.com'
},
'IMPRIMISRX' : {
'specialties' : ['Ophthalmology', 'IV Vitamin Therapy'],
'conditions' : [''],
'registered outsourcer' : True,
'accreditations' : ['VAWD', 'PCAB'],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://www.imprimisrx.com/s/'
},
'INFUSERVE AMERICA INC' : {
'specialties' : ['Clinical Trials', 'IV Vitamin Therapy'],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : ['PCAB'],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://www.infuserveamerica.com'
},
'INNOVATIVE COMPOUNDING PHARMACY' : {
'specialties' : ['Weight Loss', 'Hormone Replacement Therapy (HRT)', 'Pain Management', 'Sports Medicine', 'Veterinary', 'Ophthalmology'],
'conditions' : ['Autoimmune Disease', 'Lyme Disease', 'Alzheimers', 'Ulcerative Colitis', 'Sjorgrens Sydrome', 'Fibromyalgia', 'Traumatic Brain Injury (TBI)', 'Multiple Sclerosis', 'Crohn\'s Disease', 'Psoriasis', 'Chronic Pain', 'Hashimoto\'s', 'Parkinson\'s Disease', 'Arthritis', 'Grave\'s Disease', 'Eczema', 'Neuroinflammation', 'Irritable Bowel', 'Celiac Disease', 'Autism', 'Lupus', 'HIV/AIDs', 'Cancer', 'Seizure Disorders'],
'registered outsourcer' : False,
'accreditations' : ['PCCA', 'IACP', 'CPhA', 'PCCA', 'A4PC'],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://icpfolsom.com'
},
'ISORX RADIOPHARMACY' : {
'specialties' : ['Nuclear Medicine', 'Molecular Imaging'],
'conditions' : ['Thyroid Disease'],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'http://www.isorx.com'
},
'JUBILANT DRAXIMAGE INC DBA JUBILANT RADIOPHARMA' : {
'specialties' : ['Nuclear Medicine', 'Nuclear Cardiology', 'Molecular Imaging', 'Oncology', 'Clinical Trials'],
'conditions' : ['Cancer', 'Heart Disease', 'Lung Disease', 'Thyroid Disease', 'Bone Disease', 'Osteosarcoma', 'Chondrosarcoma', 'Ewing Sarcoma', 'Pulmonary Embolism'],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://www.jubilantradiopharma.com'
},
'LEE SILSBY COMPOUNDING PHARMACY' : {
'specialties' : [
    "Anti-Viral Medications",
    "Anti-Aging",
    "Antifungal Medications",
    "Hormone Replacement Therapy (HRT)",
    "Chelation",
    "Dentistry",
    "Dermatology",
    "Pain Management",
    "Pediatrics",
    "Podiatry",              
    "Veterinary",
    "Wellness",
    "IV Vitamin Therapy"
],
'conditions' : [
    "Chronic Fatigue Syndrome",
    "Crohn's Disease",
    "Colitis",
    "Addiction",
    "Autism",
    "Erectile Dysfunction",
    "Fibromyalgia",
    "Fissures",
    "Hirsutism",
    "Infertility",
    "Lyme Disease",
    "Methylation",
    "Nail Fungus",
    "Oral Mucositis",
    "PANDAS",
    "Rosacea",
    "Shingles",
    "Skin Lightening",
    "Warts",
    "Wound Care"
],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://leesilsby.com'
},
'LYNNFIELD DRUG INC DBA FREEDOM FERTILITY PHARMACY' : {
'specialties' : ['Fertility'],
'conditions' : ['Infertility'],
'registered outsourcer' : False,
'accreditations' : ['URAC'],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://www.freedomfertility.com'
},
'MARINER ADVANCED PHARMACY' : {
'specialties' : ['IV Antibiotic Therapy', 'Total Parenteral Nutrition (TPN)', 'Pain Management', 'Ambulatory Chemotherapy', 'Oncology', 'Clinical Trials'],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : ['PCAB', 'ACHC'],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://www.marinerpharmacy.com'
},
'MEDSPECIALTIES COMPOUNDING PHARMACY' : {
'specialties' : ['Hormone Replacement Therapy (HRT)', 'Sexual Health', 'Pain Management', 'Dermatology', 'Veterinary', 'Dentistry', 'Ophthalmology', 'IV Vitamin Therapy'],
'conditions' : ['Eczema', 'Acne'],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://www.medspecialtiesrx.com'
},
'NORTHBAY SPECIALTY PHARMACY' : {
'specialties' : ['Oncology', 'Hematology', 'Gastroenterology', 'Neurology', 'Reproductive Health'],
'conditions' : ['Rheumatoid Arthritis', 'HIV/AIDs', 'Multiple Sclerosis', 'Organ Transplant', 'Hepatitis'],
'registered outsourcer' : False,
'accreditations' : ['URAC', 'ACHC'],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://www.northbay.org/services/specialty-pharmacy/index.html'
},
"O'BRIEN PHARMACY": {
'specialties' : ['Hormone Replacement Therapy (HRT)', 'Ophthalmology', 'Gastroenterology', 'Proctology', 'Pain Management', 'Endocrinology', 'Urology', 'Dermatology', 'Pediatrics', 'Dentistry', 'Ear, Nose, and Throat'],
'conditions' : ['Corneal Scarring/Haze', 'Dry Eye', 'Cancer', 'Cataracts', 'Eye Infection'],
'registered outsourcer' : False,
'accreditations' : ['PCAB', 'PCCA', 'A4PC'],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://obrienrx.com'
},
'OMNISCRIPT COMPOUNDING LLC' : {
'specialties' : ['Hormone Replacement Therapy (HRT)', 'Fertility', 'Veterinary'],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://omniscriptrx.com'
},
'OSRX INC' : {  
'specialties' : ['Ophthalmology'],
'conditions' : [''],
'registered outsourcer' : True,
'accreditations' : ['LegitScript', 'PCAB'],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://osrxpharmaceuticals.com'
},
'OWL WESTERN PHARMACY' : {
'specialties' : ['Dermatology', 'Gastroenterology', 'Hematology', 'Immunology', 'Infectious Disease', 'Oncology', 'Rheumatology'],
'conditions' : ['HIV/AIDs', 'Hepatitis', 'Multiple Sclerosis'],
'registered outsourcer' : False,
'accreditations' : ['JCAHO'],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://www.owlspecialty.com'
},
'PACIFIC COMPOUNDING PHARMACY AND CONSULTATIONS INC' : {
'specialties' : ['Weight Loss', 'Ophthalmology', 'Dentistry', 'Oncology', 'Hormone Replacement Therapy (HRT)'],
'conditions' : ['Adrenal Fatigue'],
'registered outsourcer' : False,
'accreditations' : ['PCCA', 'A4PC', 'CPA'],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://pacificcompounding.com'
},
'PHARMA101' : {
'specialties' : [''],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://www.rxless.com/pharmacies/california/sanluisobispo/pharma101/243-Granada-Dr-Ste-A/1376998401'
},
'PHARMALABS LLC' : {
'specialties' : ['Urology', 'Wellness', 'Anti-Aging', 'Weight Loss', 'Integrative Medicine', 'Preventative Medicine', 'Functional Medicine', 'Nutritional Medicine', 'Peptide Therapy', 'Regenerative Medicine', 'Life Style Medicine'],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : ['NABP'],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://pharmalabs.com/why-choose-pharmalabs'
},
'PHARMALOGIC LOS ANGELES' : {
'specialties' : ['Nuclear Medicine', 'Molecular Imaging'],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://www.radiopharmacy.com/locations'
},
'PHARMARX PHARMACEUTICAL INC' : {
'specialties' : ['Nuclear Medicine', 'Molecular Imaging'],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://www.uppi.org'
},
'PHARMASCRIPT INC' : {
'specialties' : ['Long Term Care', 'Post-acute Care'],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://pharmscript.com'
},
'PHARMATRON A PHARMACY COMPANY' : {
'specialties' : ['Nuclear Medicine', 'Molecular Imaging'],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://www.pharmatronllc.com'
},
'PHAST PHARMACY' : {
'specialties' : ['Hospice', 'Long Term Care', 'Pain Management', 'Total Parenteral Nutrition (TPN)'],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://sites.google.com/view/phastpharmacy/home'
},
'PHILLIPS RANCH PHARMACY' : {
'specialties' : ['Wellness', 'Long Term Care'],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://sjcmcc.com'
},
'RANCHO PARK PHARMACY' : {
'specialties' : [''],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://www.ranchoparkrx.com'
},
'RANCHO SANTA FE PHARMACY' : {
'specialties' : [''],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://www.wellrx.com/find-a-pharmacy-near-me/pharmacydetail/RANCHO%20SANTA%20FE%20PHARMACY%20&%20SPIRIT%20SHOPPE/5636761/False/'
},
'RLS USA INC' : {
'specialties' : ['Nuclear Medicine', 'Molecular Imaging'],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : ''
},
'RON\'S PHARMACY SERVICES' : {
'specialties' : ['Long Term Care', 'Memory Care', 'Behavioral Health', 'Skilled Nursing'],
'conditions' : ['Intellectual & Developmental Disabilities'],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://ronspharmacyservices.com'
},
'RX UNLIMITED PHARMACY' : {
'specialties' : [''],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : ['PCCA'],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://www.rxunlimited.com'
},
'SOFIE CO' : {
'specialties' : ['Molecular Imaging', 'Nuclear Medicine'],
'conditions' : ['HIV/AIDs'],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://sofie.com'
},
'STAR PHARMACY' : {
'specialties' : ['Wellness', 'Long Term Care', ],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'http://www.starltcrx.com'
},
'STRIVE PHARMACY' : {
'specialties' : ['IV Vitamin Therapy', 'Wellness', 'Hormone Replacement Therapy (HRT)', 'Weight Loss', 'Dermatology', 'Veterinary'],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : ['LegitScript'],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://www.strivepharmacy.com'
},
'THE MEDICINE SHOPPE' : {
'specialties' : [''],
'conditions' : ['Cancer', 'Alzheimer\'s Disease', 'COPD', 'Depression', 'Congestive Heart Failure', 'Asthma', 'Osteoarthritis', 'Anxiety Disorders'],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://www.medicineshoppe.com'
},
'TMC ACQUISITION LLC DBA TAILOR MADE COMPOUNDING' : {
'specialties' : ['IV Vitamin Therapy', 'Wellness', 'Hormone Replacement Therapy (HRT)', 'Peptide Therapy', 'Weight Management', 'Anti-Aging'],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : ['NABP', 'LegitScript'],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://tailormadecompounding.com'
},
'TORMED PHARMACY' : {
'specialties' : ['Ophthalmology', 'Hormone Replacement Therapy (HRT)', 'Urology', 'Hospice'],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://www.tormedpharmacy.com'
},
'UNIVERSITY COMPOUNDING PHARMACY' : {
'specialties' : ['Weight Loss', 'Hormone Replacement Therapy (HRT)', 'IV Vitamin Therapy', 'Wellness', 'Urology', 'Dermatology','Hair Care'],
'conditions' : ['Thyroid Disease'],
'registered outsourcer' : False,
'accreditations' : ['PCAB', 'ACHC', 'A4PC'],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://www.ucprx.com/about'
},
'VALOR COMPOUNDING PHARMACY' : {
'specialties' : ['Weight Loss', 'Clinical Trials', 'Pain Management', 'Hormone Replacement Therapy (HRT)', 'Pediatrics', 'Dermatology', 'Veterinary'],
'conditions' : ['Thyroid Disease'],
'registered outsourcer' : False,
'accreditations' : ['PCAB'],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://valorcompounding.com/'
},
'VERDUGO PLAZA PHARMACY ALHAMBRA' : {
'specialties' : ['Long Term Care'],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://vpltcrx.com'
},
'VLS PHARMACY INC' : {
'specialties' : ['Ophthalmology', 'Dermatology', 'Functional Medicine', 'Integrative Medicine', 'Pain Management', 'Pediatrics'],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : ['PSSNY', 'ACAM', 'I4FM', 'A4PC', 'USPC', 'NABP'],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://newdrugloft.com'
},
'WELLNESS CENTER PHARMACY INC DBA DESIGNER DRUGS' : {
'specialties' : ['Weight Loss', 'Ophthalmology', 'Hormone Replacement Therapy (HRT)', 'Mental Health', 'Dermatology', 'Anti-Aging'],
'conditions' : ['Cataracts', 'Chronic Pain', 'Depression', 'PTSD', 'OCD', 'Anxiety', 'Substance Abuse Disorder (SUD)', 'Behavioral Addiction'],
'registered outsourcer' : False,
'accreditations' : ['MMI', 'PCCA', 'A4PC'],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://designerdrugs.us'
},
'WOMENS INTERNATIONAL PHARMACY, INC. DBA BELMAR PHARMACY' : {
'specialties' : ['Weight Loss', 'IV Vitamin Therapy', 'Hormone Replacement Therapy (HRT)', 'Anti-Aging'],
'conditions' : ['Thyroid Disease', 'Menopause', 'Vaginitis'],
'registered outsourcer' : False,
'accreditations' : ['LegitScript'],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : 'https://womensinternational.com'
},
'Next Pharmacy' : {
'specialties' : [''],
'conditions' : [''],
'registered outsourcer' : False,
'accreditations' : [''],
'facility type' : 'Sterile Compounding Pharmacy',
'url' : ''
}
   }

In [66]:
def identify_pharmacies(df: pd.DataFrame, verified_pharmacies: dict) -> pd.DataFrame:
    # Ensure columns are filled based on the verified_pharmacies dictionary
    def get_pharmacy_data(name, key):
        return verified_pharmacies.get(name, {}).get(key, None)

    df['Facility Type'] = df['Pharmacy Name'].apply(lambda name: get_pharmacy_data(name, 'facility type'))
    df['Specialties'] = df['Pharmacy Name'].apply(lambda name: ', '.join(get_pharmacy_data(name, 'specialties') or []))
    df['Conditions'] = df['Pharmacy Name'].apply(lambda name: ', '.join(get_pharmacy_data(name, 'conditions') or []))
    df['Registered Outsourcer'] = df['Pharmacy Name'].apply(lambda name: get_pharmacy_data(name, 'registered outsourcer'))
    df['Accreditations'] = df['Pharmacy Name'].apply(lambda name: ', '.join(get_pharmacy_data(name, 'accreditations') or []))
    df['URL'] = df['Pharmacy Name'].apply(lambda name: get_pharmacy_data(name, 'url'))

    return df

In [67]:
def categorize_pharmacies(df) -> pd.DataFrame:

    # Create an isGovernment column and populate it based on the License Type
    df['isGovernment'] = df['License Type'].str.contains('Government', case=False, na=False)
    
    # Create an isSatellite column and populate it based on the License Type
    df['isSatellite'] = df['License Type'].str.contains('Satellite', case=False, na=False)
    
    df = identify_hospitals(df)
    df = identify_infusion_centers(df)
    df = identify_veterinary_only(df)
    df = identify_research_centers(df)
    df = identify_pharmacies(df)
    return df

In [68]:
pharmacies = pharmacies[pharmacies['Pharmacy Name'] != "You have reached the end of your results"]
pharmacies = categorize_pharmacies(pharmacies)


# Filter rows where 'Facility Type' is either NaN or an empty string
empty_facility_type = pharmacies[pharmacies['Facility Type'].isna() | (pharmacies['Facility Type'] == '')]
# Display the result
empty_facility_type

pharmacies

TypeError: identify_pharmacies() missing 1 required positional argument: 'verified_pharmacies'

In [69]:
# File path
file_path = r"C:\Users\bvlma\PycharmProjects\California-Pharmacy-License-Dashboard\Notebooks\pharmacy_data.csv"

# Save DataFrame to CSV
pharmacies.to_csv(file_path, index=False)